In [1]:
import numpy as np
import pandas as pd

df = pd.read_pickle('df_mitdb.pkl')

In [2]:
def gustafson(signal):
    diff = []
    peaks = [0]
    for i in range(1, len(signal)-1):
        diff.append(signal[i+1] - signal[i-1])

    for i in range(0, len(diff)-3):
        if (diff[i]>0.15)&(diff[i+1]>0.15)&(diff[i+2]>0.15)&(diff[i+3]>0.15):
            if (diff[i+1]*signal[i+1] > 0) & (diff[i+2]*signal[i+2] > 0):
                if (peaks[-1] < i-10):
                    peaks.append(i)

    return peaks[1:]


def balada(signal):
    Y = []
    peaks = [0]
    for i in range(2, len(signal)-2):
        Yd = abs(signal[i+1]-signal[i-1])
        YD = abs(signal[i+2] - 2*signal[i-2])
        Y.append(1.3*Yd + 1.1*YD)


    window_size = 6
    for i in range(len(Y) - window_size + 1):
        if all(Y[j] > 1 for j in range(i, i + window_size)):
            if (peaks[-1] < i-128):
                peaks.append(i)
            # peaks.append(i)
                
    return peaks


N = int(360*0.15)
K = 6
R_si = int(360*0.24)
R_si_2 = int(360*0.12)

def Alg5_Turkish(x):

    x = np.array(x)
    # Digital FIR, y0 = x(n) - x(n-K), where K=f_sample/f_line
    K = 6
    y0 = x[K:] - x[:-K]
    
    # Squaring
    y1 = y0**2

    # Moving average
    window = np.ones(N) / N
    y2 = np.convolve(y1, window, mode='valid')

    # Normalization
    y2_min = np.min(y2)
    y2_max = np.max(y2)
    y = (y2 - y2_min) / (y2_max - y2_min)

    # adaptive tresholding
    thv = np.zeros_like(y, dtype=float)  # Initialize thv array
    
    # Calculate the treshold values
    thv[0] = 1
    for n in range(1, len(y)):
        thv[n] = ((N - 1) * thv[n - 1] + y[n]) / N

    # QRS occurs whenever signal is greater than the treshold
    temp_loc = np.where(y > thv)[0]

    R_peaks = [temp_loc[0]]

    for i in range(1, len(temp_loc)):
        if temp_loc[i] - temp_loc[i-1] > R_si:
            # I've added to every detection, so not the first value that passed treshold is considered
            R_peaks.append(temp_loc[i]+R_si + 20)
            # R_peaks.append(temp_loc[i])


    # Instead of adding 'R_si + 20' authors proposed a method for finding max (or min) value in a given window
    # to detect R-peak, but it's giving similar results and is usless from the classification point of viev

    # L = 20
    # R_peak_max = []

    # for peak in R_peaks:
    #     if np.mean(x[peak-L:peak+L]) > 0:
    #         R_peak_max.append(np.argmax(x[peak - R_si_2 : peak + R_si_2]) + peak - R_si_2)
    #     else:
    #         R_peak_max.append(np.argmin(x[peak - R_si//2 : peak + R_si//2]) + peak - R_si//2)


    # return R_peak_max

    return R_peaks



In [3]:
import numpy as np
import Algorithms as alg
import pandas as pd
import wfdb

path = 'database\RECORDS'

with open(path, 'r') as file:
    tapes = [int(line.strip()) for line in file]


df_algorithms = pd.DataFrame(index=tapes, columns=['Alg5'])

for index in df_algorithms.index:
    filename = 'database/'+ str(index)
    record = wfdb.rdrecord(filename)

    ecg = list(map(lambda x: x[0], record.p_signal))

    # df_algorithms.at[index, 'Alg1'] = np.array(alg.alg1_spanish(ecg))
    # df_algorithms.at[index, 'Alg2'] = np.array(alg.alg4_polish_20210222(ecg))
    # df_algorithms.at[index, 'Alg3'] = np.array(alg.alg3_iranian(ecg))
    # df_algorithms.at[index, 'Alg4'] = np.array(alg.alg5_pan_tompkins(ecg))
    df_algorithms.at[index, 'Alg5'] = np.array(alg.Alg5_Turkish(ecg))
    print(df_algorithms.at[index, 'Alg5'])

# name = 'df_alg5_turkish.pkl'
# df_algorithms.to_pickle(name)
# print(f"Dataframe saved as {name}")

[   305    704    987 ... 649526 649776 650031]
[   333    753   1073 ... 649040 649408 649763]
[    63    453    740 ... 649286 649596 649894]
[   201    617    918 ... 649236 649575 649916]
[   254    657    944 ... 649343 649623 649919]
[   150    498    747 ... 649511 649777 650041]
[   282    731   1100 ... 649377 649771 650028]
[   207    609    925 ... 649168 649480 649787]
[   315    752   1111 ... 649399 649742 650039]
[   274    604    841 ... 649412 649697 649966]
[   136    516    830 ... 649469 649753 650042]
[    73    424    687 ... 649405 649668 649926]
[   110    624   1008 ... 649406 649718 650036]
[   250    703   1132 ... 649559 649820 650046]
[   103    560    891 ... 649401 649691 649998]
[   218    600    877 ... 649485 649744 649999]
[   120    583    998 ... 649061 649489 649916]
[   303    712   1016 ... 649473 649782 650044]
[   244    540    968 ... 649472 649793 650022]
[   100    548    908 ... 649275 649584 649874]
[    44    365    599 ... 649420 649679 

In [213]:
# Ewaluacja
import Algorithms as alg
import time

# R_SYMBOLS = ['N', 'V', 'L', 'R', '/', 'f', 'A', 'E', 'Q', 'F', 'j', 'J', 'a', 'S', 'e']

algs = [alg.alg1_spanish, alg.alg4_polish_20210222, alg.alg3_iranian, alg.alg5_pan_tompkins, Alg5_Turkish, balada, gustafson]
# algs = [Alg5_Turkish]

for algorithm in algs:
    print(f'\nCurrent algorithm: {algorithm.__name__}')
    start_time = time.time()

    TP = 0
    FP = 0
    FN = 0

    window = (50, 50)

    all_peaks = 0
    all_detected = 0

    for tape in df.index:
        signal = df.at[tape, 'ECG']
        true_peaks = np.array(df.at[tape, 'True_peak'])

        detected = np.array(algorithm(signal))

        for peak in true_peaks:
            mask = (detected > peak-window[0]) & (detected < peak+window[0])
            is_detected = detected[mask]

            if len(is_detected) == 1:
                TP += 1
            elif len(is_detected) == 0:
                FN += 1
            else:
                TP += 1
                FP += len(is_detected)-1

        for peak in detected:
            mask = (true_peaks > peak-window[0]) & (true_peaks < peak+window[0])
            is_true = true_peaks[mask]

            if len(is_true) == 0:
                FP += 1
            
        all_peaks += len(true_peaks)
        all_detected += len(detected)

    Se = TP/(TP+FN)
    P = TP/(TP+FP)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Elapsed time:", elapsed_time, "seconds")
    print(f'All true peaks: {all_peaks}')
    print(f'All detected peaks: {all_detected}')
    print(f'TP: {TP}, FP {FP}, FN: {FN}')

    print(f'Se = {Se}, P = {P}')




Current algorithm: alg1_spanish
Elapsed time: 91.69804191589355 seconds
All true peaks: 112647
All detected peaks: 110040
TP: 109511, FP 734, FN: 3136
Se = 0.9721608209717081, P = 0.9933421016826159

Current algorithm: alg4_polish_20210222
Elapsed time: 32.00937533378601 seconds
All true peaks: 112647
All detected peaks: 109965
TP: 109643, FP 491, FN: 3004
Se = 0.9733326231501949, P = 0.9955417945411953

Current algorithm: alg3_iranian
Elapsed time: 7.083387851715088 seconds
All true peaks: 112647
All detected peaks: 108848
TP: 107911, FP 1131, FN: 4736
Se = 0.9579571582021714, P = 0.9896278498193357

Current algorithm: alg5_pan_tompkins
Elapsed time: 52.76264715194702 seconds
All true peaks: 112647
All detected peaks: 109806
TP: 109208, FP 906, FN: 3439
Se = 0.969471002334727, P = 0.9917721633943004

Current algorithm: Alg5_Turkish
Elapsed time: 29.94450354576111 seconds
All true peaks: 112647
All detected peaks: 107792
TP: 106580, FP 1605, FN: 6067
Se = 0.946141486235763, P = 0.9851